In [1]:
##### generating data to use with cellphonedb
library(Seurat)
library(CellChat)
library(patchwork)
options(stringsAsFactors = FALSE)

Attaching SeuratObject

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: ggplot2



In [2]:
pbmc <- readRDS('/home/jmitchel/data/lupus_data/lupus_subsetted_seurat_v3.rds')

# converting shorthand cell type names to full names
new_names <- sapply(as.character(pbmc@meta.data$cg_cov), function(x){
  if (x=='cM') {
    return('cMono')
  } else if (x=='ncM') {
    return('ncMono')
  } else if (x=='T4') {
    return('Th')
  } else if (x=='T8') {
    return('Tc')
  } else {
    return(x)
  }
})
names(new_names) <- NULL
pbmc@meta.data$cg_cov <- factor(new_names,levels=unique(new_names))


# subset data to SLE patients only
cells_keep <- rownames(pbmc@meta.data)[pbmc@meta.data$Status=='Managed']
pbmc <- subset(pbmc,cells = cells_keep)

# reduce to cells of the cell types that I'm using
cells_keep <- rownames(pbmc@meta.data)[pbmc@meta.data$cg_cov %in% c("B","NK","Th","Tc","cDC","cMono","ncMono")]

In [3]:
cells_keep <- sample(cells_keep,50000)
pbmc <- subset(pbmc,cells = cells_keep)
print(dim(pbmc))

[1] 32738 50000


In [4]:
pbmc@meta.data$cg_cov <- factor(pbmc@meta.data$cg_cov,levels=c("B","NK","Th","Tc","cDC","cMono","ncMono"))
pbmc@meta.data$idents <- pbmc@meta.data$cg_cov
Idents(pbmc) <- pbmc@meta.data$idents
pbmc@meta.data$cell_type <- pbmc@meta.data$cg_cov

In [5]:
write.csv(as.matrix(pbmc@assays$RNA@data), file = '/home/jmitchel/data/LR_datasets/cellphone/lupus_subsetted_cpdb_counts.csv')
write.csv(pbmc@meta.data, file = '/home/jmitchel/data/LR_datasets/cellphone/lupus_subsetted_cpdb_meta.csv')

In [11]:
# # run in terminal
# !conda activate sandbox
# !cd /home/jmitchel/data/LR_datasets/cellphone/
# !cellphonedb method statistical_analysis lupus_subsetted_cpdb_meta.csv --counts-data hgnc_symbol lupus_subsetted_cpdb_counts.csv --pvalue .05 --threads 10
